In [13]:
import geopandas as gpd
from pathlib import Path
import fiona

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

### Extraction : BDTOPO

In [25]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)

In [16]:
# Takes 50s on my laptop (old)

# Batiments
bats = gpd.read_file(GPKG, layer="batiment")
bats = bats.to_crs(4326)
bats_st_denis = gpd.sjoin(bats, saint_denis, how='inner', predicate='intersects')

# Zones d'activité (éducation)
bdedu = gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")
bdedu = bdedu.to_crs(4326)
bdedu = bdedu[bdedu["categorie"] == "Science et enseignement"]
bdedu_st_denis = gpd.sjoin(bdedu, saint_denis, how='inner', predicate='intersects')

### Extraction : Annuaire éducation

In [17]:
# Takes 12s on my laptop 

annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
annuaire_st_denis = gpd.sjoin(annuaire, saint_denis, how='inner', predicate='within')

### Extraction : Plan Cadastrale Informatisé (PCI)

In [24]:
# Takes 10s on my laptop

pci = gpd.read_file(DATA_FOLDER / "PARCELLAIRE-EXPRESS_1-1__SHP_LAMB93_D093_2024-10-01/PARCELLAIRE-EXPRESS/1_DONNEES_LIVRAISON_2024-11-00210/PEPCI_1-1_SHP_LAMB93_D093/BATIMENT.SHP")

pci = pci.to_crs(4326)
pci_st_denis = gpd.sjoin(pci, saint_denis, how='inner', predicate='intersects')

### Sauvegarder en Geopackage


In [22]:
output_gpkg = DATA_FOLDER / "saint_denis_reference_data.gpkg"

annuaire_st_denis.to_file(output_gpkg, layer='annuaire_education', driver="GPKG")
bats_st_denis.to_file(output_gpkg, layer='bdtopo_batiment', driver="GPKG")
bdedu_st_denis.to_file(output_gpkg, layer='bdtopo_education', driver="GPKG")
pci_st_denis.to_file(output_gpkg, layer='cadastre_parcellaire', driver="GPKG")

### Visualiser le package créé 

In [23]:
import geopandas as gpd
import fiona

gpkg_path = DATA_FOLDER / "saint_denis_reference_data.gpkg"
layers = fiona.listlayers(gpkg_path)

for layer in layers:
    gdf = gpd.read_file(gpkg_path, layer=layer)
    print(f"\n=== Couche: {layer} ===")
    print(f"Nombre d'objets: {len(gdf)}")
    print(f"Colonnes disponibles: {gdf.columns.tolist()}")
    print(f"Type de géométrie: {gdf.geometry.geom_type.unique()}")
    print(f"CRS: {gdf.crs}")


=== Couche: annuaire_education ===
Nombre d'objets: 107
Colonnes disponibles: ['identifiant_de_l_etablissement', 'nom_etablissement', 'type_etablissement', 'statut_public_prive', 'adresse_1', 'adresse_2', 'adresse_3', 'code_postal_left', 'code_commune', 'nom_commune', 'code_departement', 'code_academie', 'code_region', 'ecole_maternelle', 'ecole_elementaire', 'voie_generale', 'voie_technologique', 'voie_professionnelle', 'telephone', 'fax', 'web', 'mail', 'restauration', 'hebergement', 'ulis', 'apprentissage', 'segpa', 'section_arts', 'section_cinema', 'section_theatre', 'section_sport', 'section_internationale', 'section_europeenne', 'lycee_agricole', 'lycee_militaire', 'lycee_des_metiers', 'post_bac', 'appartenance_education_prioritaire', 'greta', 'siren_siret', 'nombre_d_eleves', 'fiche_onisep', 'position', 'type_contrat_prive', 'libelle_departement', 'libelle_academie', 'libelle_region', 'coordx_origine', 'coordy_origine', 'epsg_origine', 'nom_circonscription', 'latitude', 'longit